In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as utils
import matplotlib.pyplot as plt
import statistics 
from statistics import mode
from torch.optim.lr_scheduler import StepLR
%matplotlib inline

In [ ]:
def train_epoch(model, opt, criterion, batch_size=100):
    model.train()
    losses = []
    for index in range(0, x_train.shape[0], batch_size):
        x_batch = x_train[index:index + batch_size, :]
        label_batch = labels[index:index + batch_size, :]
        opt.zero_grad()
        prediction = model(x_batch)
        loss = criterion(prediction,label_batch)
        loss.backward()
        opt.step()
        losses.append(loss.item())
    losses = sum(losses)/len(losses)
    return losses

In [ ]:
def baselinePrediction(output1):
    predictions_array=[]
    for i in range(5):
        test=np.random.randint(2, size=len(x_test))
        test = list(test)
        predictions_array.append(test)
    predictions_array = np.array(predictions_array)
    print(predictions_array.shape)
    final_prediction =np.zeros(predictions_array.shape[1])
    for i in range(predictions_array.shape[1]):
        temp_list=predictions_array[:,i].tolist()
        final_prediction[i]=mode(temp_list)
    count=0
    for i in range(output1.shape[0]):
        if(output1[i]==final_prediction[i]):
            count = count+1
    print("Accuracy is ", (count/final_prediction.shape[0])*100)
    return (count/final_prediction.shape[0])*100

In [ ]:
def testingAccuracy(model,x_test,output1):
    predictions_array=[]
    model = model.eval()
    with torch.no_grad():
        predictions = model(x_test)
    test = np.asarray(predictions)
    
    test[test>0.5]=1
    test[test<=0.5]=0

    count=0
    for i in range(test.shape[0]):
        if(output1[i]==test[i]):
            count = count+1
    print("Accuracy is ", (count/test.shape[0])*100)
    return (count/test.shape[0])*100

In [ ]:
#change the dataframe to numpy array to tensor array
def changeToTensor(dataFrame):
    dataFrame= (dataFrame-dataFrame.min())/(dataFrame.max()-dataFrame.min())
    x = dataFrame.values
    x= x.astype('float64')
    x= torch.Tensor(x)
    return x

In [ ]:
def preProcess(data):
    data_numpy = data.values
    labels = data_numpy[:,-1]
    labels= labels.astype('float64')
    #labels= torch.Tensor(labels)
    data_numpy = data.drop(['date','Occupancy'],axis=1)
    features = changeToTensor(data_numpy)
    return features, labels

In [ ]:
class Net_2Layer(nn.Module):
    def __init__(self,size,H_size,H2_size):
        super(Net_2Layer, self).__init__()
        self.fc1 = nn.Linear(size,H_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(H_size,H2_size)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(H2_size,1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

In [ ]:
### create a model with pytorch#####
class Net_H(nn.Module):
    def __init__(self,size,H_size):
        super(Net_H, self).__init__()
        self.fc1 = nn.Linear(size,H_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(H_size,1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

In [ ]:
### create a model with pytorch#####
class Net_5(nn.Module):
    def __init__(self,size):
        super(Net_5, self).__init__()
        self.fc1 = nn.Linear(size,5)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(5,1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

In [ ]:
num_epochs = 100

In [ ]:
criteria= nn.BCELoss()

In [ ]:
train_data = pd.read_csv('train.txt',sep=',')
test_data = pd.read_csv('test.txt',sep=',')

In [ ]:
train_data.head()

In [ ]:
x_train, labels =preProcess(train_data)
labels_accuracy = labels
labels= torch.Tensor(labels)
labels = labels.unsqueeze(1)
print(x_train.shape, labels.shape)

In [ ]:
x_test, output = preProcess(test_data)

### Part (a) printing the loss value across 100 epochs for batch training

In [ ]:
model = Net_5(x_train.shape[1])
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [ ]:
loss_to_plot_batch=[]
for e in range(num_epochs):
    e_losses = train_epoch(model,optimizer ,criteria,x_train.shape[0])
    loss_to_plot_batch.append(e_losses)
    print("For epoch ",e+1,"loss is ",e_losses)

### Part (b) Getting the baseline accuracy for the testing data from randomly generated labels

In [ ]:
baseline_accuracy = baselinePrediction(output)

In [ ]:
baseline_error= 1-(baseline_accuracy/100)
baseline_error_plot=[baseline_error]*100

### Part (c) With the H=5,changed the learning rate and plotted training loss across every epoch. Also stopped when the error didnt decrease by much 

In [ ]:
num_epochs = 100
e_losses=[]
loss_across_epoch=[]
model = Net_5(x_train.shape[1])
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
for e in range(num_epochs):
    e_losses = train_epoch(model,optimizer ,criteria,100)
    scheduler.step()
    if(len(loss_across_epoch)>1):
        if(loss_across_epoch[-1]-e_losses)<0.0001:
            num_epochs=e
        else:
            loss_across_epoch.append(e_losses)
    else:
        loss_across_epoch.append(e_losses)
        
print("Stopped running at ",len(loss_across_epoch)," epochs rather than 100 epochs")
print("Accuracy for the testing data ")
dummy=testingAccuracy(model,x_test,output)
print("Accuracy for the training data")
dummy=testingAccuracy(model,x_train,labels_accuracy)#train_numpy[:,-1].astype('float64'))
plt.title('Training loss across epoch vs number of epochs')
plt.plot(loss_across_epoch)
baseline_error_plot1 = baseline_error_plot[:len(loss_across_epoch)]
plt.plot(baseline_error_plot1)
plt.show()

### Part (d) Plot of the training loss obtained by considering the entire training data as a batch

In [ ]:
plt.title('Batch Gradient descent method for training loss vs epoch')
plt.plot(loss_to_plot_batch)

### Part (e) Varied the number of hidden layers(H) and created a model for each of it and plotted the training and testing error

In [ ]:
x_axis=[1,2,5,10,20]
training_accuracy_every_H=[]
testing_accuracy_every_H=[]
num_epochs=100
for H in [1,2,5,10,20]:
    model = Net_H(x_train.shape[1],H)
    optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
    scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
    for e in range(num_epochs):
        e_losses = train_epoch(model,optimizer ,criteria,x_train.shape[0])
        scheduler.step()
    print("Training accuracy for H ",H)
    training_accuracy_every_H.append(testingAccuracy(model,x_train,labels_accuracy))#train_numpy[:,-1].astype('float64')))
    print("Testing accuracy for H ",H)
    testing_accuracy_every_H.append(testingAccuracy(model,x_test,output))
plt.title('Training and Testing accuracy across H vs values of H')
plt.plot(x_axis,training_accuracy_every_H)
plt.plot(x_axis,testing_accuracy_every_H)
plt.legend(['Training accuracy','Tesing accuracy'], loc='upper right')
plt.show()

### Part (f) Changed the BCELoss to MSELoss and performed the training and testing again

In [ ]:
criteria= nn.MSELoss()

In [ ]:
model = Net_5(x_train.shape[1])
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
loss_to_plot_batch=[]
for e in range(num_epochs):
    e_losses = train_epoch(model,optimizer ,criteria,x_train.shape[0])
    loss_to_plot_batch.append(e_losses)
plt.title('Batch Gradient descent method for training loss vs epoch')
plt.plot(loss_to_plot_batch)

In [ ]:
for param in model.parameters ():
    print(param.data)

In [ ]:

print("Criterai is ",criteria)

e_losses=[]
loss_across_epoch=[]
model = Net_5(x_train.shape[1])
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
for e in range(num_epochs):
    e_losses = train_epoch(model,optimizer ,criteria,100)
    scheduler.step()
    if(len(loss_across_epoch)>1):
        if(loss_across_epoch[-1]-e_losses)<0.0001:
            num_epochs=e
        else:
            loss_across_epoch.append(e_losses)
    else:
        loss_across_epoch.append(e_losses)
print("Stopped running at ",len(loss_across_epoch)," epochs instead of 100")



dummy=testingAccuracy(model,x_test,output)
print("Accuracy for the training data")
dummy=testingAccuracy(model,x_train,labels_accuracy)#train_numpy[:,-1].astype('float64'))
plt.title('Training loss across epoch vs number of epochs')

'''needed_index= 0
min_change=0.001
for i in range(1,len(loss_across_epoch)):
    if(loss_across_epoch[i-1]-loss_across_epoch[i]< min_change):
        print("Should stop at index ",i)
        needed_index = i
        break
loss_across_epoch=loss_across_epoch[:needed_index]'''
plt.plot(loss_across_epoch)    
baseline_error_plot1 = baseline_error_plot[:len(loss_across_epoch)]
plt.plot(baseline_error_plot1)
plt.show()

In [ ]:

epochs=100
x_axis=[1,2,5,10,20]
training_accuracy_every_H=[]
testing_accuracy_every_H=[]
for H in [1,2,5,10,20]:
    model = Net_H(x_train.shape[1],H)
    optimizer = torch.optim.Adam(model.parameters(),lr=0.1)
    scheduler = StepLR(optimizer, step_size=30, gamma=0.1)
    for e in range(num_epochs):
        e_losses = train_epoch(model,optimizer,criteria,x_train.shape[0])
        scheduler.step()
    training_accuracy_every_H.append(testingAccuracy(model,x_train,labels_accuracy))#train_numpy[:,-1].astype('float64')))
    testing_accuracy_every_H.append(testingAccuracy(model,x_test,output))
plt.title('Training and Testing accuracy across H vs values of H')
plt.plot(x_axis,training_accuracy_every_H)
plt.plot(x_axis,testing_accuracy_every_H)
plt.legend(['Training accuracy','Tesing accuracy'], loc='upper right')
plt.show()

### Part(g) Created a two layer network and obtained the test accuracy for every combination for H1 and H2

In [ ]:
num_epochs=100
criteria= nn.BCELoss()
for H1 in [1,2,5,10,20]:
    for H2 in [15,120,220,230]:
        print()
        print("H1 has ",H1,' neurons and H2 has',H2,' neurons')
        model = Net_2Layer(x_train.shape[1],H1,H2)
        optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
        for e in range(num_epochs):
            e_losses = train_epoch(model,optimizer ,criteria,x_train.shape[0])
        print("Testing accuracy for the 2 layer for the model with H1 ",H1," and H2 ",H2)
        testingAccuracy(model,x_test,output)
        